# Lesson 2 - ReAct pattern using langchain
- In this lesson we explain the ReAct pattern to improve the results of prompts
- Details can be found at: <https://www.promptingguide.ai/techniques/react>

> ReAct is a general paradigm that combines reasoning and acting with LLMs. ReAct prompts LLMs to generate verbal reasoning traces and actions for a task. This allows the system to perform dynamic reasoning to create, maintain, and adjust plans for acting while also enabling interaction to external environments (e.g., Wikipedia) to incorporate additional information into the reasoning.

![Diagram React](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Freact.8e7c93ae.png&w=1920&q=75)

In [1]:
# We first install the packages
%pip install langchain langchain-openai  langchain-community langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# We install our envirnment variables helper library
%pip install python-dotenv --quiet

# We load the library and the environment variables
import dotenv
dotenv.load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

This example is based on <https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/>

In [3]:
# Instead of building our prompt,
# we can reuse one of the prompts available from the Langchain Hub registry.
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [4]:
# Now we can inspect the templae
print("--> Prompt template variables:")
print(prompt.input_variables)

print("\n--> Prompt template metadata:")
print(prompt.metadata)

print("\n--> Prompt template content:")
print(prompt.template)

--> Prompt template variables:
['agent_scratchpad', 'input', 'tool_names', 'tools']

--> Prompt template metadata:
{'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}

--> Prompt template content:
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [5]:
# We load our helper callback to see what goes over the wire
from _callbacks import PrettyPrintCallbackHandler
callback = PrettyPrintCallbackHandler()

In [6]:
# We load the llm mode using our helper and based onthe settings
import _models
import _settings
llm=_models.get_llm(_settings.MODEL_NAME, callbacks=[callback])

In [7]:
# We load our challenge from the settings
import _settings
challenge_prompt=_settings.CHALLENGE_PROMPT

In [8]:
# Construct the ReAct agent
from langchain.agents import AgentExecutor, create_react_agent

# For now we don't add tools yet
tools = []
agent = create_react_agent(llm, tools, prompt)

In [9]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, 
   tools=tools, verbose=True, callbacks=[callback],
   return_intermediate_steps=True, max_iterations=15,
)

In [10]:
# Next we invoke our agent executor with our challenge
result = agent_executor.invoke({"input": challenge_prompt})
print("==================================================")


[chain][start] - inputs {'input': 'Write me a shell script to list the files in the current directory. \nOutput the script as markdown code block'}


> Entering new AgentExecutor chain...
[chat_model][start] - prompts : Answer the following questions as best you can. You have access to the following tools:



Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of []
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Write me a shell script to list the files in the current directory. 
Output the script as markdown code block
Thought:

[llm][end] - generation To answer this question, I need to write a simple shell script that lists the files in the current directory. T

In [11]:
# Let's inspect the results
print(result["input"])
print("==================================================")
steps = result["intermediate_steps"]
for step in steps:
    print(step)
    #print(step.get"log"))
    print("==================================================")
print(result["output"])

Write me a shell script to list the files in the current directory. 
Output the script as markdown code block
(AgentAction(tool='None', tool_input='None', log="To answer this question, I need to write a simple shell script that lists the files in the current directory. The script will use the 'ls' command, which is a standard command to list files in Unix and Unix-like operating systems. I will then format the script as a markdown code block.\nAction: None\nAction Input: None"), 'None is not a valid tool, try one of [].')
Here is the shell script formatted as a markdown code block:

```bash
#!/bin/bash
# This script will list the files in the current directory
ls
```
